# Deep Residual MLP with Large Ensemble

This experiment tries a fundamentally different approach:
1. Deeper architecture with residual connections [512, 256, 128, 64]
2. More aggressive regularization (dropout 0.4, weight decay 1e-3)
3. Large ensemble (10 models) with diverse architectures
4. Cosine annealing learning rate schedule
5. More epochs (400) with early stopping

Goal: Achieve breakthrough improvement, not incremental gains.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import warnings
import math
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)

# Filter DRFP to high-variance features
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange shape: {SPANGE_DF.shape}')
print(f'DRFP filtered shape: {DRFP_FILTERED.shape}')

Spange shape: (26, 13)
DRFP filtered shape: (24, 122)


In [3]:
# Combined Featurizer with Spange + DRFP + Arrhenius
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

Feature dimension: 140


In [4]:
# Residual Block for deeper networks
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.4):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.dropout(self.relu(x + self.block(x)))

# Deep Residual MLP
class DeepResidualMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[512, 256, 128, 64], output_dim=3, dropout=0.4):
        super().__init__()
        
        # Input projection
        self.input_bn = nn.BatchNorm1d(input_dim)
        self.input_proj = nn.Linear(input_dim, hidden_dims[0])
        self.input_bn2 = nn.BatchNorm1d(hidden_dims[0])
        self.input_relu = nn.ReLU()
        self.input_dropout = nn.Dropout(dropout)
        
        # Residual blocks at each hidden dimension
        self.res_blocks = nn.ModuleList()
        self.transitions = nn.ModuleList()
        
        for i, dim in enumerate(hidden_dims):
            self.res_blocks.append(ResidualBlock(dim, dropout))
            if i < len(hidden_dims) - 1:
                self.transitions.append(nn.Sequential(
                    nn.Linear(dim, hidden_dims[i+1]),
                    nn.BatchNorm1d(hidden_dims[i+1]),
                    nn.ReLU(),
                    nn.Dropout(dropout)
                ))
        
        # Output layer
        self.output = nn.Linear(hidden_dims[-1], output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # Input projection
        x = self.input_bn(x)
        x = self.input_proj(x)
        x = self.input_bn2(x)
        x = self.input_relu(x)
        x = self.input_dropout(x)
        
        # Residual blocks with transitions
        for i, res_block in enumerate(self.res_blocks):
            x = res_block(x)
            if i < len(self.transitions):
                x = self.transitions[i](x)
        
        # Output
        return self.sigmoid(self.output(x))

print('DeepResidualMLP defined')

DeepResidualMLP defined


In [5]:
# Large Ensemble Model with diverse architectures
class LargeEnsembleMLP(nn.Module):
    def __init__(self, data='single', n_models=10):
        super().__init__()
        self.data_type = data
        self.n_models = n_models
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = nn.ModuleList()
        
        # Define diverse architectures
        self.architectures = [
            {'hidden_dims': [512, 256, 128, 64], 'dropout': 0.4},  # Deep
            {'hidden_dims': [256, 128, 64], 'dropout': 0.3},       # Medium
            {'hidden_dims': [512, 256, 128], 'dropout': 0.4},      # Wide-deep
            {'hidden_dims': [128, 128, 64], 'dropout': 0.2},       # Baseline-like
            {'hidden_dims': [256, 256, 128, 64], 'dropout': 0.35}, # Balanced
            {'hidden_dims': [512, 128], 'dropout': 0.4},           # Bottleneck
            {'hidden_dims': [384, 192, 96], 'dropout': 0.35},      # Alternative
            {'hidden_dims': [256, 128, 128, 64], 'dropout': 0.3},  # Deep-narrow
            {'hidden_dims': [512, 256, 64], 'dropout': 0.4},       # Skip-like
            {'hidden_dims': [192, 192, 96, 48], 'dropout': 0.3},   # Very deep
        ]

    def train_model(self, X_train, y_train, epochs=400, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 7)  # Different seeds
            np.random.seed(42 + i * 7)
            
            arch = self.architectures[i % len(self.architectures)]
            model = DeepResidualMLP(
                input_dim, 
                hidden_dims=arch['hidden_dims'], 
                dropout=arch['dropout']
            ).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            # Stronger weight decay
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
            criterion = nn.HuberLoss()
            
            # Cosine annealing scheduler
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=50, T_mult=2, eta_min=1e-6
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step()

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

print('LargeEnsembleMLP defined')

LargeEnsembleMLP defined


In [6]:
# Run cross-validation for single solvent data
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LargeEnsembleMLP(data='single', n_models=10)
    model.train_model(train_X, train_Y, epochs=400)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [10:10<3:53:57, 610.33s/it]

  8%|▊         | 2/24 [21:41<4:01:16, 658.04s/it]

 12%|█▎        | 3/24 [32:35<3:49:34, 655.93s/it]

...

 92%|█████████▏| 22/24 [3:40:48<18:41, 560.67s/it]

 96%|█████████▌| 23/24 [3:49:57<09:17, 557.25s/it]

100%|██████████| 24/24 [4:00:09<00:00, 573.61s/it]

100%|██████████| 24/24 [4:00:09<00:00, 600.38s/it]


Single Solvent MSE: 0.051912


In [ ]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LargeEnsembleMLP(data='full', n_models=10)
    model.train_model(train_X, train_Y, epochs=400)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

In [ ]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nComparison to previous best:')
print(f'  Previous best CV: 0.010501')
print(f'  This experiment: {overall_mse:.6f}')
print(f'  Improvement: {(0.010501 - overall_mse) / 0.010501 * 100:.1f}%')

In [ ]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())